# [Commonlit Readability Price](https://www.kaggle.com/c/commonlitreadabilityprize/overview)
Hosted by - Kaggle

### Notebook 2 - inference model
uses roberta model from SimpleTransformers to predict the easeness of Readability.

### Notebook 3 - Submission 
train model without internet requirements for Submission on kaggle.


In [1]:
import numpy # linear algebra
import pandas # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm.notebook import tqdm
import os
for dirname, _, filenames in os.walk('../input'):
    for file in filenames:
        print(dirname + '/'+ file)

../input/commonlitreadabilityprize/sample_submission.csv
../input/commonlitreadabilityprize/train.csv
../input/commonlitreadabilityprize/test.csv
../input/commonlit-readability-data/wheels.zip
../input/commonlit-readability-data/__results__.html
../input/commonlit-readability-data/fsspec-2021.6.0-pyhd8ed1ab_0.tar.bz2
../input/commonlit-readability-data/requirements.txt
../input/commonlit-readability-data/__notebook__.ipynb
../input/commonlit-readability-data/__output__.json
../input/commonlit-readability-data/custom.css


# RoBERTa Regression Model

In [2]:
import pandas 
train = pandas.read_csv('../input/commonlitreadabilityprize/train.csv')
test = pandas.read_csv('../input/commonlitreadabilityprize/test.csv')

In [3]:
train_df = train[['excerpt','target']]
submit = test[['id']]
test

,id,url_legal,license,excerpt
0,c0f722661,NaN,NaN,My hope lay in Jack's promise that he would ke...
1,f0953f0a5,NaN,NaN,Dotty continued to go to Mrs. Gray's every nig...
2,0df072751,NaN,NaN,It was a bright and cheerful scene that greete...
3,04caf4e0c,https://en.wikipedia.org/wiki/Cell_division,CC BY-SA 3.0,Cell division is the process by which a parent...
4,0e63f8bea,https://en.wikipedia.org/wiki/Debugging,CC BY-SA 3.0,Debugging is the process of finding and resolv...
5,12537fe78,NaN,NaN,"To explain transitivity, let us look first at ..."
6,965e592c0,https://www.africanstorybook.org/#,CC BY 4.0,Milka and John are playing in the garden. Her ...


In [4]:
train_df.columns = ['text','labels']
test_df = list(test[['excerpt']].values.ravel())

In [5]:
train_data = train_df[:2126]
eval_data = train_df[2126:]
test_data = test_df

In [6]:
import os
import shutil
if os.path.exists('/kaggle/temp/wheels'):
    shutil.rmtree('/kaggle/temp/wheels')
if os.path.exists('/kaggle/working/wheels'):
    shutil.rmtree('/kaggle/working/wheels')
!unzip -q ../input/commonlit-readability-data/wheels.zip -d /kaggle/working/wheels 

In [7]:
!pip install \
    --requirement ../input/commonlit-readability-data/requirements.txt \
    --no-index \
    --find-links /kaggle/working/wheels

Looking in links: /kaggle/working/wheels
Processing ./wheels/simpletransformers-0.61.6-py3-none-any.whl
Processing ./wheels/datasets-1.8.0-py3-none-any.whl
Processing ./wheels/seqeval-1.2.2-py3-none-any.whl
Processing ./wheels/streamlit-0.83.0-py2.py3-none-any.whl
Processing ./wheels/tqdm-4.49.0-py2.py3-none-any.whl
Processing ./wheels/huggingface_hub-0.0.8-py3-none-any.whl
Processing ./wheels/xxhash-2.0.2-cp37-cp37m-manylinux2010_x86_64.whl
Processing ./wheels/astor-0.8.1-py2.py3-none-any.whl
Processing ./wheels/base58-2.1.0-py3-none-any.whl
Processing ./wheels/watchdog-2.1.2-py3-none-manylinux2014_x86_64.whl
Processing ./wheels/validators-0.18.2-py3-none-any.whl
Processing ./wheels/pydeck-0.6.2-py2.py3-none-any.whl
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.59.0
    Uninstalling tqdm-4.59.0:
      Successfully uninstalled tqdm-4.59.0


In [8]:
!conda install ../input/commonlit-readability-data/fsspec-2021.6.0-pyhd8ed1ab_0.tar.bz2 --offline --force-reinstall
!conda list | grep fsspec


######################################################################## | 100% 
Preparing transaction: - done
Verifying transaction: | done
Executing transaction: - \ | / - \ | / - \ | / - \ done
fsspec                    2021.6.0           pyhd8ed1ab_0    <unknown>


In [9]:
import fsspec
fsspec.__version__

'2021.06.0'

In [10]:
from simpletransformers.classification import ClassificationModel
model = ClassificationModel(
    "roberta",
    "roberta-base",
    num_labels=1,
    args={
        "regression": True,
        "num_train_epochs":10
    },
    use_cuda = True
)

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [11]:
model.train_model(train_data, eval_df=eval_data)

  0%|          | 0/2126 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/266 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/opt/conda/lib/python3.7/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


Running Epoch 1 of 10:   0%|          | 0/266 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/266 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/266 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/266 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/266 [00:00<?, ?it/s]

(2660, 0.17763262139370706)

In [12]:
import pickle
pickle.dump(model,open('roberta_model.pickle','wb'))
# model = pickle.load(open('../input/commonlit-readability/roberta_model.pickle','rb'))
predictions, raw_outputs = model.predict(test_data)
predictions

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([-0.56738281,  0.06671143, -0.38720703, -2.0234375 , -1.88183594,
       -0.90527344,  1.08300781])

In [13]:
submit['target'] = predictions
submit.to_csv('submission.csv',index = False)
submit

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,id,target
0,c0f722661,-0.567383
1,f0953f0a5,0.066711
2,0df072751,-0.387207
3,04caf4e0c,-2.023438
4,0e63f8bea,-1.881836
5,12537fe78,-0.905273
6,965e592c0,1.083008
